In [ ]:
!pip install -q groq langchain langchain-community langchain-groq chromadb pypdf openpyxl pandas gradio sentence-transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 2.5 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 32.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 137.5/137.5 kB 6.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.7/21.7 MB 61.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 328.2/328.2 kB 18.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 278.2/278.2 kB 14.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 51.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 39.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 103.3/103.3 kB 4.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.4/17.4 MB 65.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.5/72.5 kB 4.0 MB/s eta 0:00

In [ ]:
# Install required packages
!pip install -q groq langchain langchain-community langchain-groq chromadb pypdf openpyxl pandas gradio sentence-transformers langchain-text-splitters langchain-core

import gradio as gr
import os
import tempfile
from typing import List, Tuple
import pandas as pd
from groq import Groq

# LangChain imports
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_community.document_loaders import PyPDFLoader, TextLoader
from langchain_community.vectorstores import Chroma
from langchain_community.embeddings import HuggingFaceEmbeddings
from langchain_core.documents import Document

class RAGSystem:
    def __init__(self):
        self.vectorstore = None
        self.groq_client = None
        self.embeddings = HuggingFaceEmbeddings(
            model_name="sentence-transformers/all-MiniLM-L6-v2"
        )
        self.text_splitter = RecursiveCharacterTextSplitter(
            chunk_size=1000,
            chunk_overlap=200,
            length_function=len
        )

    def verify_api_key(self, api_key: str) -> Tuple[bool, str]:
        """Verify if the Groq API key is valid"""
        try:
            client = Groq(api_key=api_key)
            # Test with a simple completion
            response = client.chat.completions.create(
                model="llama-3.3-70b-versatile",
                messages=[{"role": "user", "content": "Hi"}],
                max_tokens=10
            )
            self.groq_client = client
            return True, "✅ API Key is valid and working!"
        except Exception as e:
            return False, f"❌ API Key verification failed: {str(e)}"

    def load_pdf(self, file_path: str) -> List[Document]:
        """Load and process PDF files"""
        loader = PyPDFLoader(file_path)
        documents = loader.load()
        return self.text_splitter.split_documents(documents)

    def load_text(self, file_path: str) -> List[Document]:
        """Load and process text files"""
        loader = TextLoader(file_path)
        documents = loader.load()
        return self.text_splitter.split_documents(documents)

    def load_excel(self, file_path: str) -> List[Document]:
        """Load and process Excel files"""
        df = pd.read_excel(file_path)
        # Convert dataframe to text
        text_content = df.to_string()
        doc = Document(page_content=text_content, metadata={"source": file_path})
        return self.text_splitter.split_documents([doc])

    def process_documents(self, files: List) -> str:
        """Process uploaded documents and create vector store"""
        if not files:
            return "❌ No files uploaded!"

        all_documents = []
        processed_files = []

        for file in files:
            file_path = file.name
            file_ext = os.path.splitext(file_path)[1].lower()

            try:
                if file_ext == '.pdf':
                    docs = self.load_pdf(file_path)
                elif file_ext == '.txt':
                    docs = self.load_text(file_path)
                elif file_ext in ['.xlsx', '.xls']:
                    docs = self.load_excel(file_path)
                else:
                    continue

                all_documents.extend(docs)
                processed_files.append(os.path.basename(file_path))
            except Exception as e:
                return f"❌ Error processing {os.path.basename(file_path)}: {str(e)}"

        if not all_documents:
            return "❌ No valid documents found to process!"

        # Create vector store
        try:
            self.vectorstore = Chroma.from_documents(
                documents=all_documents,
                embedding=self.embeddings,
                collection_name="rag_collection"
            )
            return f"✅ Successfully processed {len(processed_files)} file(s):\n" + "\n".join(f"  • {f}" for f in processed_files) + f"\n\nTotal chunks created: {len(all_documents)}"
        except Exception as e:
            return f"❌ Error creating vector store: {str(e)}"

    def query_documents(self, question: str, api_key: str) -> str:
        """Query the documents using RAG"""
        if not api_key:
            return "❌ Please provide a valid Groq API key first!"

        if not self.groq_client:
            success, message = self.verify_api_key(api_key)
            if not success:
                return message

        if not self.vectorstore:
            return "❌ Please upload and process documents first!"

        if not question.strip():
            return "❌ Please enter a question!"

        try:
            # Retrieve relevant documents
            relevant_docs = self.vectorstore.similarity_search(question, k=4)

            # Prepare context from retrieved documents
            context = "\n\n".join([doc.page_content for doc in relevant_docs])

            # Create prompt
            prompt = f"""Based on the following context, please answer the question accurately and concisely. If the answer cannot be found in the context, please say so.\n\nContext:\n{context}\n\nQuestion: {question}\n\nAnswer:"""

            # Query Groq
            response = self.groq_client.chat.completions.create(
                model="llama-3.3-70b-versatile",
                messages=[
                    {"role": "system", "content": "You are a helpful assistant that answers questions based on provided context. Be accurate and cite specific information from the context when possible."},
                    {"role": "user", "content": prompt}
                ],
                temperature=0.3,
                max_tokens=1024
            )

            answer = response.choices[0].message.content

            # Add sources
            sources = set([doc.metadata.get('source', 'Unknown') for doc in relevant_docs])
            sources_text = "\n\n📚 **Sources:**\n" + "\n".join(f"  • {os.path.basename(s)}" for s in sources)

            return answer + sources_text

        except Exception as e:
            return f"❌ Error during query: {str(e)}"

# Initialize RAG system
rag_system = RAGSystem()

# Create Gradio interface
with gr.Blocks(title="RAG with Groq LLM", theme=gr.themes.Soft()) as demo:
    gr.Markdown("""
    # 📚 RAG Application with Groq LLM
    Upload your documents (PDF, Excel, Text) and ask questions!
    """)

    with gr.Row():
        with gr.Column(scale=1):
            api_key_input = gr.Textbox(
                label="🔑 Groq API Key",
                type="password",
                placeholder="Enter your Groq API key here..."
            )
            verify_btn = gr.Button("✓ Verify API Key", variant="secondary")
            api_status = gr.Textbox(label="API Key Status", interactive=False)

            gr.Markdown("---")

            file_upload = gr.File(
                label="📎 Upload Documents",
                file_count="multiple",
                file_types=[".pdf", ".txt", ".xlsx", ".xls"]
            )
            process_btn = gr.Button("📤 Process Documents", variant="primary")
            process_status = gr.Textbox(label="Processing Status", interactive=False)

        with gr.Column(scale=2):
            chatbot = gr.Chatbot(label="💬 Chat History", height=400)
            question_input = gr.Textbox(
                label="❓ Ask a Question",
                placeholder="Enter your question here...",
                lines=2
            )
            submit_btn = gr.Button("🚀 Submit Question", variant="primary")
            clear_btn = gr.Button("🗑️ Clear Chat", variant="secondary")

    gr.Markdown("""
    ### 📝 Instructions:
    1. Enter your Groq API key and verify it
    2. Upload one or more documents (PDF, Excel, or Text files)
    3. Click "Process Documents" to prepare the knowledge base
    4. Ask questions about your documents!
    """)

    # Event handlers
    def verify_api(api_key):
        success, message = rag_system.verify_api_key(api_key)
        return message

    def process_files(files):
        return rag_system.process_documents(files)

    def respond(message, chat_history, api_key):
        if not message.strip():
            return chat_history, ""

        bot_message = rag_system.query_documents(message, api_key)
        chat_history.append((message, bot_message))
        return chat_history, ""

    def clear_chat():
        return [], ""

    verify_btn.click(verify_api, inputs=[api_key_input], outputs=[api_status])
    process_btn.click(process_files, inputs=[file_upload], outputs=[process_status])
    submit_btn.click(respond, inputs=[question_input, chatbot, api_key_input], outputs=[chatbot, question_input])
    question_input.submit(respond, inputs=[question_input, chatbot, api_key_input], outputs=[chatbot, question_input])
    clear_btn.click(clear_chat, outputs=[chatbot, question_input])

# Launch the app
demo.launch(share=True, debug=True)

/tmp/ipython-input-291764297.py:22: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the `langchain-huggingface package and should be used instead. To use it run `pip install -U `langchain-huggingface` and import as `from `langchain_huggingface import HuggingFaceEmbeddings``.
  self.embeddings = HuggingFaceEmbeddings(
/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

/tmp/ipython-input-291764297.py:159: DeprecationWarning: The 'theme' parameter in the Blocks constructor will be removed in Gradio 6.0. You will need to pass 'theme' to Blocks.launch() instead.
  with gr.Blocks(title="RAG with Groq LLM", theme=gr.themes.Soft()) as demo:
/tmp/ipython-input-291764297.py:186: UserWarning: You have not specified a value for the `type` parameter. Defaulting to the 'tuples' format for chatbot messages, but this is deprecated and will be removed in a future version of Gradio. Please set type='messages' instead, which uses openai-style dictionaries with 'role' and 'content' keys.
  chatbot = gr.Chatbot(label="💬 Chat History", height=400)
/tmp/ipython-input-291764297.py:186: DeprecationWarning: The default value of 'allow_tags' in gr.Chatbot will be changed from False to True in Gradio 6.0. You will need to explicitly set allow_tags=False if you want to disable tags in your chatbot.
  chatbot = gr.Chatbot(label="💬 Chat History", height=400)


Colab notebook detected. This cell will run indefinitely so that you can see errors and logs. To turn off, set debug=False in launch().
* Running on public URL: https://058334f3c9b73a286d.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
